<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/sketch/justin/GETSTATS_Clean_CH_2019_Social_Distancing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clean Chicago 2019 Social Distancing Dataset

**Instructions:**

1. Mount the google drive to access the data by following **Steps to Mount the Drive**.
2. Provide the drive file path to the data, city name, and year in the fourth code cell. 

**Steps to Mount the Drive:**

1. Execute the second code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.

In [ ]:
pip install geopandas

In [ ]:
import os
import geopandas
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

import altair as alt
from vega_datasets import data

# # Allow altair to visualize large datasets
# alt.data_transformers.disable_max_rows()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
city = 'Chicago'
city_directory = 'ch'
year = '2019'

# Provide a list of CSV file paths to read
path = Path('drive/MyDrive/big-data-project/data/clean-data')
path_list = [path/city_directory/'social'/year]

In [ ]:
#Appending into dataframe

def get_df(path_list):
    df = []
    first = True
    for directory in tqdm(path_list, desc='Years'):
        f_list = os.listdir(directory)
        csv_files = [directory/f for f in f_list if f.endswith('.csv')]
        for csv in tqdm(csv_files, desc='Days'):
            if first:
                df = pd.read_csv(csv, dtype={'cbg': object})
                first = False
            else:
                df = df.append(pd.read_csv(csv, dtype={'cbg': object}), ignore_index=True)
    return df.sort_values(by=['date_range_start'])

In [ ]:
%%time

df = get_df(path_list)



CPU times: user 8min 34s, sys: 11.3 s, total: 8min 46s
Wall time: 17min 42s


In [ ]:
df

,Unnamed: 0,cbg,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,mean_home_dwell_time,mean_non_home_dwell_time,mean_distance_traveled_from_home
1997,66069,170319900000,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,29,2733,"{""16001-50000"":4,""0"":5,"">50000"":5,""2001-8000"":...","{""16001-50000"":169,"">50000"":525,""2001-8000"":14...",4,0,"{""721-1080"":1,""<60"":19,""361-720"":1,""61-360"":3}","[3,2,3,4,3,1,3,2,5,1,3,1,3,1,1,2,1,2,1,1,3,3,3,1]",1,1,"{""170310307011"":1,""170438411023"":1,""1703143060...",1,133,93,"{""21-45"":4,""541-600"":1,""721-840"":1,""301-360"":1...",0,"{""0-25"":17,""76-100"":6}",126,271,2733
0,79,170311201001,2019-01-01T00:00:00-06:00,2019-01-02T00:00:00-06:00,98,763,"{""16001-50000"":9,""0"":26,"">50000"":16,""2001-8000...","{""16001-50000"":245,"">50000"":130,""<1000"":85,""20...",26,694,"{""721-1080"":8,""361-720"":16,""61-360"":12,""<60"":2...","[43,46,50,47,49,51,50,49,44,47,41,39,47,43,44,...",7,7,"{""170318066002"":1,""170311504013"":1,""5512700160...",1,165,157,"{""21-45"":3,""481-540"":3,""541-600"":1,""721-840"":3...",73,"{""0-25"":25,""76-100"":44,""51-75"":12,""26-50"":17}",710,384,20411
4445,147814,170312503004,2019-01-01T00:00:00-06:00,2019-01-02T00:00:00-06:00,37,4726,"{""0"":20,""2001-8000"":8,""1-1000"":1,""1001-2000"":2...","{""1001-2000"":32,""2001-8000"":39,""8001-16000"":12...",19,817,"{""721-1080"":5,""361-720"":2,""61-360"":6,""<60"":2,""...","[20,23,21,20,22,20,20,21,23,24,23,18,16,19,14,...",1,1,"{""170318066002"":1,""170318435001"":1,""1703107060...",2,0,128,"{""21-45"":2,""46-60"":1,""721-840"":3,""301-360"":2,""...",100,"{""0-25"":4,""76-100"":24,""51-75"":2,""26-50"":3}",792,126,4958
4444,147633,171978835211,2019-01-01T00:00:00-06:00,2019-01-02T00:00:00-06:00,60,4018,"{""16001-50000"":8,""0"":24,"">50000"":4,""2001-8000""...","{""16001-50000"":40,"">50000"":14,""<1000"":76,""2001...",25,999,"{""721-1080"":4,""361-720"":8,""61-360"":8,""<60"":8,""...","[28,36,35,31,35,32,38,34,33,36,38,34,36,36,33,...",3,1,"{""471570114001"":1,""170318300051"":1,""1703182411...",1,20,157,"{""21-45"":7,""541-600"":1,""46-60"":1,""721-840"":4,""...",98,"{""0-25"":6,""76-100"":38,""51-75"":1,""26-50"":6}",810,199,45463
4443,147630,170898534004,2019-01-01T00:00:00-06:00,2019-01-02T00:00:00-06:00,83,641,"{""16001-50000"":3,""0"":45,"">50000"":1,""2001-8000""...","{""16001-50000"":372,"">50000"":6,""<1000"":189,""200...",44,779,"{""721-1080"":18,""361-720"":9,""61-360"":12,""<60"":1...","[32,33,41,46,43,50,47,42,47,47,40,45,46,42,36,...",5,1,"{""170898536003"":2,""170438465192"":1,""1708985300...",1,0,285,"{""21-45"":2,""481-540"":1,""46-60"":3,""721-840"":2,""...",100,"{""0-25"":13,""76-100"":60,""51-75"":7,""26-50"":1}",698,129,1051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425785,74212,170318024021,2019-12-31T00:00:00-06:00,2020-01-01T00:00:00-06:00,97,2009,"{""16001-50000"":19,""0"":27,"">50000"":5,""2001-8000...","{""16001-50000"":65,"">50000"":55,""<1000"":29,""2001...",27,657,"{""721-1080"":19,""361-720"":14,""61-360"":9,""<60"":2...","[53,61,54,53,55,56,56,57,44,38,38,33,41,34,36,...",12,5,"{""170318077003"":1,""170978645152"":1,""1703180240...",4,101,191,"{""21-45"":6,""541-600"":1,""46-60"":4,""721-840"":1,""...",85,"{""0-25"":25,""76-100"":53,""51-75"":11,""26-50"":8}",637,246,4116
2425784,74018,181270510062,2019-12-31T00:00:00-06:00,2020-01-01T00:00:00-06:00,85,3168,"{""16001-50000"":8,""0"":23,"">50000"":5,""2001-8000""...","{""16001-50000"":90,"">50000"":138,""<1000"":14,""200...",25,801,"

In [ ]:
# Selecting the columsn that we need for the analysis
# col = ['date_range_start',
#        'cbg',
#        'device_count',  # Total active devices
#        'completely_home_device_count',
#        'part_time_work_behavior_devices',
#        'full_time_work_behavior_devices',
#        'delivery_behavior_devices',
#        'median_percentage_time_home',
#        'median_home_dwell_time',
#        'mean_home_dwell_time',
#        'median_non_home_dwell_time',
#        'mean_non_home_dwell_time',
#        'distance_traveled_from_home', # Median Distance traveled
#        'mean_distance_traveled_from_home']
col = ['date_range_start',
       'cbg',
       'device_count',  # Total active devices
       'completely_home_device_count',
       'part_time_work_behavior_devices',
       'full_time_work_behavior_devices']
devices_mobility = df[col]

In [ ]:
#Renaming columns

devices_mobility = devices_mobility.rename(columns={'date_range_start':'date','device_count':'total', 'completely_home_device_count':'completely_home','part_time_work_behavior_devices':'part_time_work', 'full_time_work_behavior_devices':'full_time_work'})
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work
1997,2019-01-01T00:00:00-05:00,170319900000,29,4,1,1
0,2019-01-01T00:00:00-06:00,170311201001,98,26,7,7
4445,2019-01-01T00:00:00-06:00,170312503004,37,19,1,1
4444,2019-01-01T00:00:00-06:00,171978835211,60,25,3,1
4443,2019-01-01T00:00:00-06:00,170898534004,83,44,5,1
...,...,...,...,...,...,...
2425785,2019-12-31T00:00:00-06:00,170318024021,97,27,12,5
2425784,2019-12-31T00:00:00-06:00,181270510062,85,25,8,8
2425783,2019-12-31T00:00:00-06:00,170438401034,111,29,10,6
2425793,2019-12-31T00:00:00-06:00,171978802032,124,36,9,7


In [ ]:
# devices_mobility

In [ ]:
#Listing out the device count columns

device_columns = ['completely_home','part_time_work','full_time_work']

In [ ]:
#Calculating percentage for all the device columns

for column in device_columns :
    devices_mobility['percentage_'+column]= devices_mobility.apply(lambda row: row[column]/row['total']*100.0 if row['total']!=0.0 else 0.0, axis=1)
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work
1997,2019-01-01T00:00:00-05:00,170319900000,29,4,1,1,13.793103,3.448276,3.448276
0,2019-01-01T00:00:00-06:00,170311201001,98,26,7,7,26.530612,7.142857,7.142857
4445,2019-01-01T00:00:00-06:00,170312503004,37,19,1,1,51.351351,2.702703,2.702703
4444,2019-01-01T00:00:00-06:00,171978835211,60,25,3,1,41.666667,5.000000,1.666667
4443,2019-01-01T00:00:00-06:00,170898534004,83,44,5,1,53.012048,6.024096,1.204819
...,...,...,...,...,...,...,...,...,...
2425785,2019-12-31T00:00:00-06:00,170318024021,97,27,12,5,27.835052,12.371134,5.154639
2425784,2019-12-31T00:00:00-06:00,181270510062,85,25,8,8,29.411765,9.411765,9.411765
2425783,2019-12-31T00:00:00-06:00,170438401034,111,29,10,6,26.126126,9.009009,5.405405
2425793,2019-12-31T00:00:00-06:00,171978802032,124,36,9,7,29.032258,7.258065,5.645161


In [ ]:
#Normalising the percentage columns

for column in device_columns:
        devices_mobility['norm_'+column] = (devices_mobility['percentage_'+ column] - devices_mobility['percentage_'+column].mean()) / devices_mobility['percentage_'+column].std()

devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
1997,2019-01-01T00:00:00-05:00,170319900000,29,4,1,1,13.793103,3.448276,3.448276,-1.735035,-1.064748,-0.535610
0,2019-01-01T00:00:00-06:00,170311201001,98,26,7,7,26.530612,7.142857,7.142857,-0.448142,-0.353529,0.240316
4445,2019-01-01T00:00:00-06:00,170312503004,37,19,1,1,51.351351,2.702703,2.702703,2.059541,-1.208274,-0.692194
4444,2019-01-01T00:00:00-06:00,171978835211,60,25,3,1,41.666667,5.000000,1.666667,1.081080,-0.766036,-0.909779
4443,2019-01-01T00:00:00-06:00,170898534004,83,44,5,1,53.012048,6.024096,1.204819,2.227324,-0.568894,-1.006775
...,...,...,...,...,...,...,...,...,...,...,...,...
2425785,2019-12-31T00:00:00-06:00,170318024021,97,27,12,5,27.835052,12.371134,5.154639,-0.316352,0.652933,-0.177244
2425784,2019-12-31T00:00:00-06:00,181270510062,85,25,8,8,29.411765,9.411765,9.411765,-0.157054,0.083244,0.716826
2425783,2019-12-31T00:00:00-06:00,170438401034,111,29,10,6,26.126126,9.009009,5.405405,-0.489008,0.005712,-0.124579
2425793,2019-12-31T00:00:00-06:00,171978802032,124,36,9,7,29.032258,7.258065,5.645161,-0.195396,-0.331351,-0.074226


In [ ]:
#Conversion to datetime format

devices_mobility['date'] = pd.to_datetime(devices_mobility['date'],utc= True)
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
1997,2019-01-01 05:00:00+00:00,170319900000,29,4,1,1,13.793103,3.448276,3.448276,-1.735035,-1.064748,-0.535610
0,2019-01-01 06:00:00+00:00,170311201001,98,26,7,7,26.530612,7.142857,7.142857,-0.448142,-0.353529,0.240316
4445,2019-01-01 06:00:00+00:00,170312503004,37,19,1,1,51.351351,2.702703,2.702703,2.059541,-1.208274,-0.692194
4444,2019-01-01 06:00:00+00:00,171978835211,60,25,3,1,41.666667,5.000000,1.666667,1.081080,-0.766036,-0.909779
4443,2019-01-01 06:00:00+00:00,170898534004,83,44,5,1,53.012048,6.024096,1.204819,2.227324,-0.568894,-1.006775
...,...,...,...,...,...,...,...,...,...,...,...,...
2425785,2019-12-31 06:00:00+00:00,170318024021,97,27,12,5,27.835052,12.371134,5.154639,-0.316352,0.652933,-0.177244
2425784,2019-12-31 06:00:00+00:00,181270510062,85,25,8,8,29.411765,9.411765,9.411765,-0.157054,0.083244,0.716826
2425783,2019-12-31 06:00:00+00:00,170438401034,111,29,10,6,26.126126,9.009009,5.405405,-0.489008,0.005712,-0.124579
2425793,2019-12-31 06:00:00+00:00,171978802032,124,36,9,7,29.032258,7.258065,5.645161,-0.195396,-0.331351,-0.074226


In [ ]:
# Normalize time

devices_mobility['date'] = devices_mobility['date'].dt.normalize()
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
1997,2019-01-01 00:00:00+00:00,170319900000,29,4,1,1,13.793103,3.448276,3.448276,-1.735035,-1.064748,-0.535610
0,2019-01-01 00:00:00+00:00,170311201001,98,26,7,7,26.530612,7.142857,7.142857,-0.448142,-0.353529,0.240316
4445,2019-01-01 00:00:00+00:00,170312503004,37,19,1,1,51.351351,2.702703,2.702703,2.059541,-1.208274,-0.692194
4444,2019-01-01 00:00:00+00:00,171978835211,60,25,3,1,41.666667,5.000000,1.666667,1.081080,-0.766036,-0.909779
4443,2019-01-01 00:00:00+00:00,170898534004,83,44,5,1,53.012048,6.024096,1.204819,2.227324,-0.568894,-1.006775
...,...,...,...,...,...,...,...,...,...,...,...,...
2425785,2019-12-31 00:00:00+00:00,170318024021,97,27,12,5,27.835052,12.371134,5.154639,-0.316352,0.652933,-0.177244
2425784,2019-12-31 00:00:00+00:00,181270510062,85,25,8,8,29.411765,9.411765,9.411765,-0.157054,0.083244,0.716826
2425783,2019-12-31 00:00:00+00:00,170438401034,111,29,10,6,26.126126,9.009009,5.405405,-0.489008,0.005712,-0.124579
2425793,2019-12-31 00:00:00+00:00,171978802032,124,36,9,7,29.032258,7.258065,5.645161,-0.195396,-0.331351,-0.074226


In [ ]:
# Check the number of rows for each CBG

print('Min: {}\nMax: {}'.format(min(set(devices_mobility.groupby('cbg').count()['date'])), max(set(devices_mobility.groupby('cbg').count()['date']))))

Min: 2
Max: 365


In [ ]:
for col in devices_mobility.columns:
    print('\n')
    print(col)
#     if devices_mobility[col].dtype
#     print(devices_mobility[col].dtype)
    print('Min: {}\nMax: {}'.format(devices_mobility[col].min(), devices_mobility[col].max()))



date
Min: 2019-01-01 00:00:00+00:00
Max: 2019-12-31 00:00:00+00:00


cbg
Min: 170310101001
Max: 550599900000


total
Min: 5
Max: 9089


completely_home
Min: 1
Max: 3637


part_time_work
Min: 1
Max: 504


full_time_work
Min: 1
Max: 244


percentage_completely_home
Min: 0.6944444444444444
Max: 94.87179487179486


percentage_part_time_work
Min: 0.1718213058419244
Max: 66.66666666666666


percentage_full_time_work
Min: 0.06067961165048543
Max: 57.14285714285714


norm_completely_home
Min: -3.0584153064913604
Max: 6.456488060129292


norm_part_time_work
Min: -1.6954772012255084
Max: 11.105009681422562


norm_full_time_work
Min: -1.2470644256122123
Max: 10.741186523936774


In [ ]:
# Print all Null and NaN values

print('We have {} null values.'.format(len(devices_mobility[devices_mobility.isna().any(axis=1)])))

We have 0 null values.


In [ ]:
# Get average values for all columns

In [ ]:
# Save Dataframe to file
# Do same for all cities
# Combine entries to get average for 2019

In [ ]:
# # Grouping based on day

# grouped_dm = devices_mobility.groupby(devices_mobility['date']).mean()
# grouped_dm

In [ ]:
# grouped_dm.reset_index()

In [ ]:
# #Visualising mobility

# title = '{} {}'.format(city,year)
# alt.Chart(grouped_dm.reset_index()).mark_line().encode(
#     x=alt.X('date', title='Date'),
#     y=alt.Y(alt.repeat('row'), type='quantitative')
# ).properties(
#     width=300,
#     height=250
# ).repeat(
#     row=['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']
# ).interactive()

![ch-daily-2019.png](https://github.com/chouhandiksha/bigdataproject/raw/main/media/social-dist/ch-daily-2019.png)

In [ ]:
# #Adding month column for further usage

# devices_mobility['month'] = devices_mobility['date'].dt.month
# devices_mobility

In [ ]:
# #Grouping data on basis of month value
# grouped_dm = devices_mobility.groupby([devices_mobility['month']]).mean()
# grouped_dm

In [ ]:
# d = []
# for col in ['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']:
#         for m in range(1,13):
#             d.append({'month': m, 'column': col, 'value': grouped_dm[col][m]})
# d = alt.Data(values=d)

In [ ]:
# title = '{} {}'.format(city,year)
# alt.Chart(d, title=title).mark_line().encode(
#     x=alt.X('month:O', title='Month'),
#     y=alt.Y('value:Q', type='quantitative', title='Percentage'),
#     color='column:N'
# ).properties(
#     width=300,
#     height=250
# ).interactive()

![ch-monthly-2019](https://github.com/chouhandiksha/bigdataproject/raw/main/media/social-dist/ch-monthly-2019.png)

In [ ]:
df_summary = devices_mobility.describe()
df_summary

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
count,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06
mean,9.504381e+01,2.983398e+01,8.549604e+00,5.511380e+00,3.096627e+01,8.979337e+00,5.998589e+00,-3.668441e-13,1.189522e-12,-6.917475e-14
std,1.119792e+02,4.251987e+01,1.018083e+01,7.564772e+00,9.897878e+00,5.194712e+00,4.761510e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,5.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.944444e-01,1.718213e-01,6.067961e-02,-3.058415e+00,-1.695477e+00,-1.247064e+00
25%,5.200000e+01,1.500000e+01,3.000000e+00,1.000000e+00,2.413793e+01,4.918033e+00,2.150538e+00,-6.898786e-01,-7.818151e-01,-8.081579e-01
50%,7.200000e+01,2.200000e+01,6.000000e+00,3.000000e+00,3.043478e+01,8.333333e+00,4.597701e+00,-5.369663e-02,-1.243580e-01,-2.942109e-01
75%,1.030000e+02,3.300000e+01,1.100000e+01,7.000000e+00,3.723404e+01,1.223404e+01,8.661417e+00,6.332446e-01,6.265420e-01,5.592402e-01
max,9.089000e+03,3.637000e+03,5.040000e+02,2.440000e+02,9.487179e+01,6.666667e+01,5.714286e+01,6.456488e+00,1.110501e+01,1.074119e+01


In [ ]:
filename = 'drive/MyDrive/big-data-project/sketch/justin/{}_social_summary_{}.csv'.format(city_directory, year)
df_summary.to_csv(filename)